In [1]:
pip install google-search-results


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=bd0a06cc979076da9c85156ed31994f9529203a7c3c8ee08c39e29a96d7b8200
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [2]:
!pip install --quiet google-search-results


In [3]:
from serpapi import GoogleSearch
print("OK")


OK


In [4]:
import os
os.environ["SERPAPI_API_KEY"] = "67f82ab9a77a1d7dc650a629566120e6179574d62836eb4039bf7080e71fd2b6"
print("Loaded:", os.getenv("SERPAPI_API_KEY"))


Loaded: 67f82ab9a77a1d7dc650a629566120e6179574d62836eb4039bf7080e71fd2b6


In [5]:
from serpapi import GoogleSearch
import os

print("SERPAPI_API_KEY =", os.getenv("SERPAPI_API_KEY"))

params = {
    "engine": "google",
    "q": "transformer models agriculture",
    "api_key": os.getenv("SERPAPI_API_KEY"),
}
search = GoogleSearch(params)
results = search.get_dict()

print("Keys:", results.keys())
print("Organic results:", len(results.get("organic_results", [])))
print("Errors:", results.get("error"))


SERPAPI_API_KEY = 67f82ab9a77a1d7dc650a629566120e6179574d62836eb4039bf7080e71fd2b6
Keys: dict_keys(['search_metadata', 'search_parameters', 'search_information', 'related_questions', 'ai_overview', 'organic_results', 'scholarly_articles', 'pagination', 'serpapi_pagination'])
Organic results: 10
Errors: None


In [6]:
# Install required libraries
# ----------------------
!pip install --quiet langgraph==1.0.3
!pip install --quiet langchain-openai wikipedia arxiv serpapi tiktoken graphviz

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.8 MB/s eta 0:00:00


In [12]:
# Load Secrets (Colab)
# ----------------------
from google.colab import userdata
import os, json, time, logging, re
logging.basicConfig(level=logging.INFO)

def load_keys():
    # If running in interactive Colab with userdata configured, this will populate env vars.
    # Otherwise, set them via os.environ or Colab secrets manager as needed.
    try:
        os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY") or os.getenv("AZURE_OPENAI_API_KEY","")
        os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT") or os.getenv("AZURE_OPENAI_ENDPOINT","")
        os.environ["OPENAI_API_VERSION"] = userdata.get("OPENAI_API_VERSION") or os.getenv("OPENAI_API_VERSION","")
        os.environ["AZURE_OPENAI_DEPLOYMENT"] = userdata.get("AZURE_OPENAI_DEPLOYMENT") or os.getenv("AZURE_OPENAI_DEPLOYMENT","")
        #os.environ["SERPAPI_API_KEY"] = userdata.get("SERPAPI_API_KEY") or os.getenv("SERPAPI_API_KEY","")
    except Exception as e:
        logging.warning("Could not load userdata keys: %s", e)

load_keys()
print("Azure key present:", bool(os.getenv("AZURE_OPENAI_API_KEY")))
#print("SerpAPI key present:", bool(os.getenv("SERPAPI_API_KEY")))


Azure key present: True


In [44]:
# LLM Setup (Azure GPT-4.1 or user-provided wrapper)
# ----------------------
# NOTE: ensure your env vars are set. The wrapper usage may vary by version; we call via llm_call wrapper.
from langchain_openai import AzureChatOpenAI
try:
    llm = AzureChatOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        api_version=os.getenv("OPENAI_API_VERSION"),
        model="gpt-4.1",
        temperature=0.2
    )
except Exception as e:
    logging.warning("AzureChatOpenAI init failed: %s. Proceeding; LLM calls may fail.", e)
    llm = None

In [55]:
# Token encoder and constants
# ----------------------
try:
    enc = tiktoken.get_encoding("cl100k_base")
except Exception:
    # fallback to a naive encoder (rare)
    enc = None

MAX_TOKENS = 12000

def count_tokens(text):
    if not text:
        return 0
    try:
        if enc:
            return len(enc.encode(text))
        return len(text.split())
    except Exception:
        return len(text.split())

# ----------------------
# Robust helpers: LLM wrapper and token-safe memory append
# ----------------------
def llm_call(prompt, max_retries=2):
    """
    Robust wrapper to call the LLM. Supports multiple common wrapper interfaces.
    Returns text (str) or an error note.
    """
    if llm is None:
        return "[LLM NOT CONFIGURED]"
    for attempt in range(1, max_retries + 1):
        try:
            # Try common interfaces
            if hasattr(llm, "invoke"):
                resp = llm.invoke(prompt)
                text = getattr(resp, "content", str(resp))
            elif callable(llm):
                resp = llm(prompt)
                # resp might be text or object
                text = resp if isinstance(resp, str) else getattr(resp, "content", str(resp))
            elif hasattr(llm, "generate"):
                gen = llm.generate([prompt])
                text = gen.generations[0][0].text
            else:
                raise RuntimeError("Unrecognized LLM interface")
            return str(text).strip()
        except Exception as e:
            logging.warning("LLM call failed (attempt %d/%d): %s", attempt, max_retries, e)
            if attempt == max_retries:
                return f"[LLM ERROR after {max_retries} attempts: {e}]"
            time.sleep(1 + attempt)

def safe_append_tokens(history, new_text, max_tokens=MAX_TOKENS):
    """
    Append new_text to history while ensuring token count <= max_tokens.
    If needed, truncate older tokens from history (keep the most recent).
    """
    if not history:
        return new_text or ""
    try:
        if enc:
            h_tokens = enc.encode(history)
            n_tokens = enc.encode(new_text)
            if len(h_tokens) + len(n_tokens) <= max_tokens:
                return history + "\n\n" + new_text
            allowed_history = max_tokens - len(n_tokens) - 50  # margin
            if allowed_history <= 0:
                return new_text
            kept = h_tokens[-allowed_history:]
            kept_text = enc.decode(kept)
            return kept_text + "\n\n[TRUNCATED DUE TO TOKEN LIMIT]\n\n" + new_text
    except Exception:
        pass
    # Fallback: simple char-based truncation
    max_chars = 100000
    combined = history + "\n\n" + new_text
    return combined[-max_chars:]

In [56]:
# State schema
# ----------------------
class ResearchState(BaseModel):
    user_prompt: str = ""
    plan: str = ""
    research_notes: str = ""
    draft: str = ""
    final_output: str = ""
    bibliography: list = Field(default_factory=list)  # list of dicts {title, link, type}

    plan_feedback: str = ""
    final_feedback: str = ""
    plan_approved: bool = False
    final_approved: bool = False

    memory: str = ""
    iteration_count: int = 0

# ----------------------
# Global behavior flags
# Set HIL_MODE to "interactive" for Colab UI, or "auto" for automated runs.
# ----------------------
HIL_MODE = os.getenv("HIL_MODE", "interactive")  # "interactive" or "auto"


In [71]:
def planner_node(state: ResearchState):
    prompt = f"""
You are a Planner Agent.
Decompose the task into a structured research plan as JSON with keys:
- title (string)
- sections: list of objects {{heading: str, objective: str}}
- search_queries: list of strings (search queries to run)
- comparison_tasks: list of strings

User Prompt: {state.user_prompt}
Previous feedback: {state.plan_feedback}

Return only valid JSON.
"""
    out = llm_call(prompt)

    # Ensure structured JSON with defaults
    try:
        parsed = json.loads(out)
        parsed.setdefault("title", state.user_prompt[:50])
        parsed.setdefault("sections", [{"heading": "Introduction", "objective": "Provide context"}])
        parsed.setdefault("search_queries", [state.user_prompt])
        parsed.setdefault("comparison_tasks", [])
        state.plan = json.dumps(parsed, indent=2)
    except Exception:
        # Fallback: wrap raw output in JSON
        state.plan = json.dumps({
            "title": state.user_prompt[:50],
            "sections": [{"heading": "Generated Plan", "objective": out}],
            "search_queries": [state.user_prompt],
            "comparison_tasks": []
        }, indent=2)

    state.memory = safe_append_tokens(state.memory, "[PLAN]\n" + state.plan)
    return state


In [75]:
# Plan review (HIL Node 1)
def hil_plan_review(state: ResearchState):
    print("\n==== GENERATED PLAN ====\n")
    print(state.plan)
    if HIL_MODE == "interactive":
        decision = input("\nApprove plan? (yes/no): ").strip().lower()
        if decision.startswith("y"):
            state.plan_approved = True
            state.plan_feedback = ""
        else:
            state.plan_approved = False
            state.plan_feedback = input("Provide feedback for Planner (used in next iteration): ")
    else:
        # Auto: accept only if no feedback exists
        state.plan_approved = (state.plan_feedback == "")
    return state


In [72]:
def researcher_node(state: ResearchState):
    notes = ""
    sources = []
    logging.info("Researcher: running research for prompt: %s", state.user_prompt)

    # Use planner search queries if available
    queries = []
    try:
        p = json.loads(state.plan)
        if isinstance(p, dict) and "search_queries" in p:
            queries = p.get("search_queries", [])
    except Exception:
        pass
    if not queries:
        queries = [state.user_prompt]

    # --- Wikipedia search robust ---
    import wikipedia
    for q in queries[:3]:
        try:
            wiki_summary = wikipedia.summary(q, sentences=5, auto_suggest=True, redirect=True)
            notes += f"\n### Wikipedia Summary: {q}\n{wiki_summary}\n"
            wiki_link = f"https://en.wikipedia.org/wiki/{q.replace(' ', '_')}"
            sources.append({"title": f"Wikipedia: {q}", "link": wiki_link, "type": "Wikipedia", "query": q})
        except wikipedia.DisambiguationError as e:
            options = e.options[:3]
            notes += f"\n[Wikipedia disambiguation for '{q}', options considered: {options}]\n"
        except Exception:
            notes += f"\n[Wikipedia summary not found or failed for query: {q}]\n"

    # --- arXiv search ---
    import arxiv
    try:
        for q in queries[:3]:
            search = arxiv.Search(query=q, max_results=3)
            got = False
            for result in search.results():
                got = True
                notes += f"\n### arXiv: {result.title}\n{result.summary}\n"
                sources.append({"title": result.title, "link": result.entry_id, "type": "arXiv", "query": q})
            if not got:
                notes += f"\n[arXiv: no results for query: {q}]\n"
    except Exception as e:
        notes += f"\n[arXiv search failed: {e}]\n"

    # --- SerpAPI search ---
    try:
        from serpapi import GoogleSearch
        serp_key = os.getenv("SERPAPI_API_KEY", "")
        if serp_key:
            for q in queries[:3]:
                params = {"engine": "google", "q": q, "api_key": serp_key}
                search = GoogleSearch(params)
                raw = search.get_dict()
                results = raw.get("organic_results") or []
                for res in results[:5]:
                    title = res.get("title") or "No title"
                    link = res.get("link") or res.get("source") or "No link"
                    notes += f"\n### Web: {title}\nLink: {link}\n"
                    sources.append({"title": title, "link": link, "type": "Web", "query": q})
        else:
            notes += "\n[SerpAPI API key not configured — skipping web search]\n"
    except Exception as e:
        notes += f"\n[SerpAPI search failed: {e}]\n"

    # Save notes and sources
    state.research_notes = notes
    state.bibliography = sources
    state.memory = safe_append_tokens(state.memory, "[RESEARCH]\n" + notes)
    return state


In [73]:
def writer_node(state: ResearchState):
    if not state.bibliography:
        state.bibliography = [{"title": "No sources found", "link": "", "type": "None"}]

    citation_map = "\n".join([f"[{i+1}] {s['title']}: {s.get('link','')}" for i, s in enumerate(state.bibliography)])
    prompt = f"""
You are a Writer Agent.
Using the PLAN and RESEARCH NOTES, write a polished Markdown literature review.

PLAN:
{state.plan}

RESEARCH NOTES:
{state.research_notes}

Include in-text citations like [1], [2], … corresponding to the References section below.
Add a References section at the end with all sources and clickable links:

{citation_map}

Return only the Markdown draft.
"""
    out = llm_call(prompt)
    state.draft = out
    state.memory = safe_append_tokens(state.memory, "[DRAFT]\n" + out)
    return state


In [74]:
def reviewer_node(state: ResearchState):
    bib_count = len(state.bibliography)
    draft = state.draft or ""
    cited_indices = set(int(m) for m in re.findall(r"\[(\d+)\]", draft))
    missing = [i for i in cited_indices if i < 1 or i > bib_count]

    review_note = ""
    if missing:
        review_note += f"Found citations that do not map to bibliography entries: {missing}\n"

    prompt = f"""
You are a Reviewer Agent. Check the draft for:
- Hallucinated citations (citations present in the text that are not in the bibliography)
- Structural coherence and relevance to the user prompt
- Any major missing elements based on the plan

Draft:
{draft}

Bibliography items (count {bib_count}):
{json.dumps(state.bibliography, indent=2)}

Respond with a short verdict: either "GOOD" or "BAD" and then a brief justification (1-3 sentences).
"""
    llm_decision = llm_call(prompt)
    normalized = (llm_decision or "").strip().upper()
    approved = ("GOOD" in normalized) and not missing
    state.final_approved = approved

    if not approved:
        review_note += f"LLM reviewer message:\n{llm_decision}\n"
        state.plan_feedback = safe_append_tokens(state.plan_feedback or "", review_note)

    return state


In [76]:
# Final review (HIL Node 2)
def hil_final_review(state: ResearchState):
    print("\n==== DRAFT =====\n")
    print(state.draft or "[No draft produced]")
    if HIL_MODE == "interactive":
        dec = input("\nAccept final draft? (yes/no): ").strip().lower()
        if dec.startswith("y"):
            state.final_approved = True
            state.final_output = state.draft
        else:
            state.final_approved = False
            state.final_feedback = input("Provide revision feedback: ")
            state.plan_feedback = state.final_feedback
    else:
        # Auto: accept only if reviewer approved
        if state.final_approved:
            state.final_output = state.draft
        else:
            state.final_feedback = "Auto-reject: reviewer failed. Improve citations & specificity."
            state.plan_feedback = state.final_feedback
    return state


In [77]:
# Workflow graph construction
# ----------------------
workflow = StateGraph(ResearchState)
workflow.add_node("planner", planner_node)
workflow.add_node("hil_plan", hil_plan_review)
workflow.add_node("researcher", researcher_node)
workflow.add_node("writer", writer_node)
workflow.add_node("reviewer", reviewer_node)
workflow.add_node("hil_final", hil_final_review)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "hil_plan")
workflow.add_conditional_edges(
    "hil_plan", lambda s: "researcher" if s.plan_approved else "planner",
    {"researcher": "researcher", "planner": "planner"}
)
workflow.add_edge("researcher", "writer")
workflow.add_edge("writer", "reviewer")
workflow.add_conditional_edges(
    "reviewer", lambda s: "hil_final" if s.final_approved else "writer",
    {"hil_final": "hil_final", "writer": "writer"}
)
workflow.add_conditional_edges(
    "hil_final", lambda s: END if s.final_approved else "planner",
    {END: END, "planner": "planner"}
)

app = workflow.compile()


In [79]:
# Run Workflow
# ----------------------
MAX_ITER = 50
initial = ResearchState(user_prompt="Impact of transformer models on 2024 agriculture analytics")
iteration = 0
final_state = None
trace = []

for state_dict in app.stream(initial):
    iteration += 1
    # ensure iteration_count saved
    state_dict['iteration_count'] = iteration
    # keep minimal state for printing and trace (avoid saving large memory)
    state_to_print = {k: v for k, v in state_dict.items() if k != "memory"}
    print(f"\n--- Iteration {iteration} ---")
    print("State:", state_to_print)
    trace.append(state_to_print)
    final_state = state_dict
    if iteration >= MAX_ITER:
        print("\n⚠ Reached max iterations. Stopping.")
        break


--- Iteration 1 ---
State: {'planner': {'user_prompt': 'Impact of transformer models on 2024 agriculture analytics', 'plan': '{\n  "title": "Impact of Transformer Models on 2024 Agriculture Analytics",\n  "sections": [\n    {\n      "heading": "Introduction to Transformer Models",\n      "objective": "Explain the fundamentals of transformer models and their relevance to data analytics."\n    },\n    {\n      "heading": "Current Applications in Agriculture Analytics",\n      "objective": "Identify and describe how transformer models are being used in agriculture analytics as of 2024."\n    },\n    {\n      "heading": "Benefits and Improvements",\n      "objective": "Analyze the advantages transformer models bring to agricultural data analysis, including accuracy, scalability, and automation."\n    },\n    {\n      "heading": "Challenges and Limitations",\n      "objective": "Discuss the challenges faced when implementing transformer models in agricultural contexts, such as data require

/tmp/ipython-input-1598106229.py:37: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():



--- Iteration 7 ---
State: {'researcher': {'user_prompt': 'Impact of transformer models on 2024 agriculture analytics', 'plan': '{\n  "title": "Impact of Transformer Models on 2024 Agriculture Analytics",\n  "sections": [\n    {\n      "heading": "Introduction to Transformer Models",\n      "objective": "Explain the fundamentals of transformer models and their relevance to data analytics."\n    },\n    {\n      "heading": "Applications of Transformer Models in Agriculture Analytics",\n      "objective": "Identify and describe key use cases of transformer models in agricultural data analysis in 2024."\n    },\n    {\n      "heading": "Efficiency of Transformer Models Compared to Traditional Methods",\n      "objective": "Analyze and compare the efficiency of transformer models versus traditional machine learning approaches in agriculture analytics."\n    },\n    {\n      "heading": "Challenges and Limitations",\n      "objective": "Discuss the challenges, limitations, and potential bar

In [80]:
from graphviz import Digraph
# Workflow Diagram
# ----------------------
dot = Digraph()
dot.attr(rankdir="LR")
nodes = ["planner","hil_plan","researcher","writer","reviewer","hil_final"]
for n in nodes: dot.node(n, n.upper())
dot.edge("planner","hil_plan")
dot.edge("hil_plan","researcher", label="approve")
dot.edge("hil_plan","planner", label="reject")
dot.edge("researcher","writer")
dot.edge("writer","reviewer")
dot.edge("reviewer","hil_final", label="good")
dot.edge("reviewer","writer", label="bad")
dot.edge("hil_final","planner", label="reject")
dot.edge("hil_final", END, label="accept")
dot.render("workflow_graph", format="png", cleanup=True)
print("Rendered workflow_graph.png in working directory.")

Rendered workflow_graph.png in working directory.


In [82]:
# Save artifacts and execution trace (concise confirmation)
try:
    with open("/content/research_trace.json", "w") as f:
        json.dump(trace, f, indent=2)
    if final_state and final_state.get("final_output"):
        with open("/content/final_draft.md", "w") as f:
            f.write(final_state["final_output"])
    elif final_state and final_state.get("draft"):
        with open("/content/latest_draft.md", "w") as f:
            f.write(final_state["draft"])

    # Always print concise confirmation
    print("Saved trace and draft files to /content/(research_trace.json, final_draft.md or latest_draft.md).")

except Exception as e:
    logging.warning("Could not save artifacts: %s", e)


Saved trace and draft files to /content/(research_trace.json, final_draft.md or latest_draft.md).
